In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export core
%nbdev_default_class_level 3

Cells will be exported to geomechy.core,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import numpy as np
from geomechy.base import ItemDict
from geomechy.utils import *
from geomechy.io import jsonReader
from geomechy.materials import *

# Core
> Main program

## Material Object

In [ ]:
%nbdev_export
class MaterialSet(dict):
    
    def __init__(self, material):
        
        fluids = material["Fluids"]
        media  = material["PorousMedia"]
        temp   = material["Temp"]
        
        for idx, f in enumerate(fluids):
            if f["Type"] == "Water":
                w = Water(f)
                self[f"water_{idx+1}"] = w
            elif f["Type"] == "Oil":
                o = Oil(f)
                self[f"oil_{idx+1}"] = o
            elif f["Type"] == "Gas":
                g = Gas(f)
                self[f"gas_{idx+1}"] = g
            elif f["Type"] == "Air":
                a = Air(f)
                self[f"air_{idx+1}"] = a
            else:
                pass
                
        for idx, m in enumerate(media):
            if m["Type"] == "Soil":
                s = Soil(m)
                self[f"soil_{idx+1}"] = s
            elif m["Type"] == "Rock":
                r = Rock(m)
                self[f"rock_{idx+1}"] = r
            else:
                pass  

In [ ]:
jsonFile = "./assets/json/template.json"
data = jsonReader(jsonFile)
props = data["Materials"]
#props
mat = MaterialSet(props)
print(mat)

{'water_1': <geomechy.materials.Water object at 0x000001545DC19C50>, 'soil_1': <geomechy.materials.Soil object at 0x000001545DC19B00>}


## Node Set Object

In [ ]:
%nbdev_export
class NodeSet(ItemDict):
    
    def getNodeCoords(self, nodeIds):
        return np.array(self.get(nodeIds))
        
    def readFromDict(self, data):
        nodes_dict = data["Nodes"]
        node_dim = nodes_dict["dim"]
        
        for node in nodes_dict["coords"]:
            self.add(node[0], node[1:])
        

## Element Set Object

In [ ]:
%nbdev_export
class ElementSet(ItemDict):
    
    def __init__(self, nodes, props):
        ItemDict.__init__(self)
        self.nodes  = nodes
        self.props  = props
        self.groups = {}
    
    def __iter__(self):
        elements = []

        for groupName in self.iterGroupNames():
            for element in self.iterElementGroup(groupName):
                elements.append(element)

        return iter(elements)
    
    
    def getDofTypes(self):
        dofTypes = []
        
        for element in self:
            for dofType in element.dofTypes:
                if dofType not in dofTypes:
                    dofTypes.append(dofType)
        return dofTypes
    
    
    def readFromFile(self, fileName):
        fin = open(filename)
        
    
    def add(self, ID, modelName, elementNodes):  

        #Check if the model exists
        if not hasattr(self.props, modelName):
            RuntimeError('Missing properties for model ' + modelName)

        modelProps = getattr(self.props, modelName)

        
        #Check if the model has a type
        if not hasattr( modelProps, 'type' ):
            RuntimeError('Missing type for model ' + modelName)

        modelType = getattr(modelProps, 'type')

        
        #Load the element
        exec(f'from geomechy.elements import {modelType} as element',globals())

        
        #Create the element
        elem = element(elementNodes, modelProps)

        #Check if the node IDs are valid
        for nodeID in elem.getNodes():
            if not nodeID in self.nodes:
                raise RuntimeError('Node ID ' + str(nodeID) + ' does not exist')

        
        #Add the element to the element set
        ItemDict.add(self, ID, elem)

        #Add the element to the correct group
        self.addToGroup(modelName, ID)
    
    
    def addToGroup(self, modelType, ID):

        if not modelType in self.groups:
            self.groups[modelType] = [ID]
        else:
            self.groups[modelType].append( ID )

    def iterGroupNames(self):
        return self.groups
        
    def iterElementGroup (self, groupName):
        if groupName == "All":
            return iter(self)
        else:
            return iter(self.get(self.groups[groupName]))

## Constitutive Manager

In [ ]:
%nbdev_export
class ConstitutiveManager(list):
    
    def __init__(self, matProps):
        matType = matProps.type
        exec(f'from geomechy.materials import {matType} as material', globals())
        self.mat = material(matProps)
        self.iIter = -1

    def reset(self):
        self.iIter = -1

    def getStress(self, kinematic, iSam=-1):

        if iSam == -1:
            self.iIter += 1
            iSam = self.iIter

        self.mat.setIter(iSam)
        return self.mat.getStress(kinematic)

    def commitHistory(self):
        self.mat.commitHistory()

In [ ]:
%nbdev_hide
notebook2script()

NameError: name 'notebook2script' is not defined